In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('./ML DATASET/loan.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## Preprocessing 

In [4]:
# Check the missing values
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
# Dropping Loan_ID as it is not an important field
df = df.drop(['Loan_ID'],axis = 1)

In [6]:
# Drop the missing values
df_d = df.dropna()

In [7]:
df_d.shape

(480, 12)

In [8]:
df_d.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

## Converting Categorical Values

In [9]:
df_d['Dependents'].value_counts()

Dependents
0     274
2      85
1      80
3+     41
Name: count, dtype: int64

In [10]:
df_d['Dependents'] = df_d['Dependents'].replace(to_replace = '3+', value=4)
df_d['Dependents'].value_counts()

/tmp/ipykernel_14340/3938076158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['Dependents'] = df_d['Dependents'].replace(to_replace = '3+', value=4)


Dependents
0    274
2     85
1     80
4     41
Name: count, dtype: int64

In [11]:
df_d['Property_Area'].value_counts()

Property_Area
Semiurban    191
Urban        150
Rural        139
Name: count, dtype: int64

In [12]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 1 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             480 non-null    object 
 1   Married            480 non-null    object 
 2   Dependents         480 non-null    object 
 3   Education          480 non-null    object 
 4   Self_Employed      480 non-null    object 
 5   ApplicantIncome    480 non-null    int64  
 6   CoapplicantIncome  480 non-null    float64
 7   LoanAmount         480 non-null    float64
 8   Loan_Amount_Term   480 non-null    float64
 9   Credit_History     480 non-null    float64
 10  Property_Area      480 non-null    object 
 11  Loan_Status        480 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 48.8+ KB


In [13]:
if(df['Married'].dtypes=='O'): print('yes')

yes


In [14]:
df_d.replace({'Gender':{'Male':1,'Female':0},'Married':{'Yes':1,'No':0},'Self_Employed':{'No':0,'Yes':1},'Education':{'Graduate':1,'Not Graduate':0},'Property_Area':{'Rural':1,'Urban':0,'Semiurban':3},'Dependents':{'0':0,'1':1,'2':2,'4':3}},inplace=True)   

/tmp/ipykernel_14340/1258654571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d.replace({'Gender':{'Male':1,'Female':0},'Married':{'Yes':1,'No':0},'Self_Employed':{'No':0,'Yes':1},'Education':{'Graduate':1,'Not Graduate':0},'Property_Area':{'Rural':1,'Urban':0,'Semiurban':3},'Dependents':{'0':0,'1':1,'2':2,'4':3}},inplace=True)


In [15]:
#Training and testing sets 
x = df_d.drop(columns = ['Loan_Status'])
y = df_d['Loan_Status']

In [16]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 1 to 613
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             480 non-null    int64  
 1   Married            480 non-null    int64  
 2   Dependents         480 non-null    int64  
 3   Education          480 non-null    int64  
 4   Self_Employed      480 non-null    int64  
 5   ApplicantIncome    480 non-null    int64  
 6   CoapplicantIncome  480 non-null    float64
 7   LoanAmount         480 non-null    float64
 8   Loan_Amount_Term   480 non-null    float64
 9   Credit_History     480 non-null    float64
 10  Property_Area      480 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 45.0 KB


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 5)  

## SVM Model 

### Linear

In [18]:
classifier = svm.SVC(kernel='linear')
classifier.fit(x_train,y_train)

SVC(kernel='linear')

In [21]:
#Training accuracy
x_train_pred = classifier.predict(x_train)
train_acc = accuracy_score(x_train_pred,y_train)
train_acc

0.7942708333333334

In [23]:
#Test accuracy
x_test_pred = classifier.predict(x_test)
test_acc = accuracy_score(x_test_pred, y_test)
test_acc

0.7916666666666666

### Poly

In [24]:
classifier_poly = svm.SVC(kernel='poly')
classifier_poly.fit(x_train,y_train)

SVC(kernel='poly')

In [27]:
#Train accuracy
x_train_poly = classifier_poly.predict(x_train)
train_accuracy = accuracy_score(x_train_poly,y_train)
train_accuracy


0.703125

In [28]:
#Test accuracy
x_test_poly = classifier_poly.predict(x_test)
test_accuracy = accuracy_score(x_test_poly, y_test)
test_accuracy

0.6666666666666666